<h1> Материалы курса "Анализ данных в инфромационной безопасности" </h1>

<h2> Практическое занятие № 2.1 </h2>

<h2> Тема: "Python+SQLite" </h2>

<h2>Автор:</h2>

<p>Синадский Алексей</p>
<p>Исследователь исследовательского центра UDV Group</p>
<p>aleksei.sinadskiy@udv.group</p>


Содержание:
- Чтение данных из Excel
- Создание БД
- Проектирование и создание таблиц
- Вставка данных
- Получение данных
- Анализ имеющихся данных
- Построение модели прогнозирования средствами SQL

# Импорт нужных модулей

In [10]:
import pandas as pd  # для обработки данных в Python
import sqlite3  # лёгкая БД - подробнее посмотреть можно тут: https://habr.com/ru/articles/754400/
import contextlib  # для удобного открытия и закрытия подключеия к БД

# Чтение и запись данных XLSX

## Чтение

Путь к файлу

In [11]:
xlsx_file_path = './Автосалон_амбарная_книга.xlsx'
sheet_name = 'Sheet1'

Прочитайте данные из файла с помощью Pandas

In [12]:
df = pd.read_excel(xlsx_file_path, sheet_name=sheet_name)

Посмотрите результат

In [13]:
df

,Worker,Car,Result
0,Андрей 20 лет холост,Lada 21063 VIN 1,Заработал 117$
1,Никита 25 лет женат,Ford Sierra VIN 2,Заработал 87$
2,Иван 35 лет холост,ИЖ Ода VIN 3,Заработал 100
3,Андрей 20 лет холост,Nissan Pulsar VIN 4,Потерял 91$
4,Никита 25 лет женат,Nissan Pulsar VIN 5,Потерял 126$
...,...,...,...
295,Никита 25 лет женат,Peugeot 206 VIN 296,Заработал 455$
296,Иван 35 лет холост,Ford Sierra VIN 297,Заработал 415$
297,Андрей 20 лет холост,Peugeot 206 VIN 298,Заработал 395$
298,Никита 25 лет женат,Москвич 412 VIN 299,Заработал 364


## Запись

Запишите данные в файл XLSX с другим названием

In [14]:
df.to_excel('output.xlsx', sheet_name=sheet_name, index=False)

## Предобработка

Выделите марку машины и её идентификатор в отдельные поля. Столбцы назовите `car_model` и `car_vin`

In [15]:
df['car_model'] = df.apply(lambda row: row.Car.split('VIN')[0], axis=1)
df['car_vin'] = df.apply(lambda row: row.Car.split('VIN')[1], axis=1)

Посмотрите результат

In [16]:
df.head(5)

,Worker,Car,Result,car_model,car_vin
0,Андрей 20 лет холост,Lada 21063 VIN 1,Заработал 117$,Lada 21063,1
1,Никита 25 лет женат,Ford Sierra VIN 2,Заработал 87$,Ford Sierra,2
2,Иван 35 лет холост,ИЖ Ода VIN 3,Заработал 100,ИЖ Ода,3
3,Андрей 20 лет холост,Nissan Pulsar VIN 4,Потерял 91$,Nissan Pulsar,4
4,Никита 25 лет женат,Nissan Pulsar VIN 5,Потерял 126$,Nissan Pulsar,5


Выделите результат продажи в виде числа в формате float.

Напишите функцию, которая по значению столбца Result возвращает численное значение, а затем примените её в `map`

In [17]:
def user_readable_result_to_numbers(result_str):
    """
    Возваращает численное значение результата продажи по строковому из амбарной книги

    Parameters
    ----------
    result_str : str
        Строковое описание результата продажи из амбарной книги

    Returns
    -------
    numeric_result : float
        Численное значение результата продажи

    Raises
    ------
    ValueError
        Вызывается в случае, если встречено не интерпретируемое значение
    """
    numeric_result = 0
    direction, value_str = result_str.split(' ')
    abs_value = float(value_str.rstrip("$"))
    if direction == "Заработал":
      numeric_result = abs_value
    elif direction == "Потерял":
      numeric_result = -abs_value
    else:
      raise ValueError("Unexpected format of data in treding results")
    return numeric_result

df['trade_result'] = df['Result'].map(user_readable_result_to_numbers)

In [18]:
df

,Worker,Car,Result,car_model,car_vin,trade_result
0,Андрей 20 лет холост,Lada 21063 VIN 1,Заработал 117$,Lada 21063,1,117.0
1,Никита 25 лет женат,Ford Sierra VIN 2,Заработал 87$,Ford Sierra,2,87.0
2,Иван 35 лет холост,ИЖ Ода VIN 3,Заработал 100,ИЖ Ода,3,100.0
3,Андрей 20 лет холост,Nissan Pulsar VIN 4,Потерял 91$,Nissan Pulsar,4,-91.0
4,Никита 25 лет женат,Nissan Pulsar VIN 5,Потерял 126$,Nissan Pulsar,5,-126.0
...,...,...,...,...,...,...
295,Никита 25 лет женат,Peugeot 206 VIN 296,Заработал 455$,Peugeot 206,296,455.0
296,Иван 35 лет холост,Ford Sierra VIN 297,Заработал 415$,Ford Sierra,297,415.0
297,Андрей 20 лет холост,Peugeot 206 VIN 298,Заработал 395$,Peugeot 206,298,395.0
298,Никита 25 лет женат,Москвич 412 VIN 299,Заработал 364,Москвич 412,299,364.0


Выделите сотрудников в отдельный датафрейм. В нём должны быть колонки `name`, `age`, `marital_status`

In [19]:
def user_readable_worker_to_details(worker_str):
    """
    Возвращает информационные поля по строковому описанию сотрудника из амбарной книги

    Parameters
    ----------
    worker_str : str
        Описание сотрудника из амбарной книги

    Returns
    -------
    tuple
        Набор свойств сотрдуника: имя, возраст, семейное положение
    """
    name, age, _, marital_status = worker_str.split(' ')
    return name, age, marital_status

df_workers = pd.DataFrame(columns=['name', 'age', 'marital_status'])
df_workers[['name', 'age', 'marital_status']] = df['Worker'].map (user_readable_worker_to_details).apply(pd.Series)
df_workers = df_workers.drop_duplicates()  # удаление полных дубликатов

In [20]:
df_workers

,name,age,marital_status
0,Андрей,20,холост
1,Никита,25,женат
2,Иван,35,холост


# Проектирование и создание таблиц

Общий синтаксис:
```
connection = sqlite3.connect('my_database.db')
cursor = connection.cursor()
cursor.execute('SQL statement')
connection.commit()
connection.close()
```
Для автозакрытия можно использовать
```with contextlib.closing(sqlite3.connect(path_to_file)) as conn```

Для автокоммита:
```with contextlib.closing(conn.cursor()) as cursor```

В итоге с автозакрытием подключения и автокоммитом:
```
with contextlib.closing(sqlite3.connect(path_to_file)) as conn:
    with contextlib.closing(conn.cursor()) as cursor:
        cursor.execute(SQL statement)
```

Не требуется в каждой записи хранить информацию о сотруднике - вынесем его в отдельную таблицу.

Также не будем хранить старые поля - сохраним только вновь выделенные.

Создадим две таблицы: "Сотрудники" и "Продажи".

В таблице "Сотрудники" создайте поля "Имя", "Возраст", "Семейное положение"

In [21]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Workers (
              id INTEGER PRIMARY KEY,
              name TEXT NOT NULL,
              age INTEGER,
              marital_status TXT
            )
            '''
        )

В таблице "Продажи" создайте поля "Модель машины", "Идентификатор машины", "Результат продажи", "Идентификатор продавца"

In [22]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Sales (
              id INTEGER PRIMARY KEY,
              car_model TEXT NOT NULL,
              car_vin TEXT NOT NULL,
              trade_result REAL NOT NULL,
              worker_id INT NOT NULL
            )
            '''
        )


# Вставка данных

Добавьте данные о сотрудниках из `df_workers` в таблицу `Workers` БД

In [25]:
data = list(zip(df_workers['name'], df_workers['age'], df_workers['marital_status']))  # преобразуем столбцы DataFrame в кортежи

with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    cursor = connection.cursor()
    cursor.executemany(  # executemany - для набора данных
        'INSERT INTO Workers (name, age, marital_status) VALUES (?, ?, ?)', data

    )
    connection.commit()
data


[('Андрей', '20', 'холост'),
 ('Никита', '25', 'женат'),
 ('Иван', '35', 'холост')]

Добавьте данные о продажах в таблицу `Sales` (для каждой продажи нужно добавить ссылку на сотрудника в таблице сотрудников)

In [24]:
workers_saved = list(df_workers.name + ' ' + df_workers.age + ' лет ' + df_workers.marital_status)  # список всех работников, по которому можно будет выбрать индекс

data = list(zip(df['car_model'], df['car_vin'], df['trade_result'],
                [workers_saved.index(worker_str) + 1 for worker_str in df.Worker]))

with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    cursor = connection.cursor()
    cursor.executemany(
        'INSERT INTO Sales (car_model, car_vin, trade_result, worker_id) VALUES (?, ?, ?, ?)', data
    )
    connection.commit()
data

[('Lada 21063 ', ' 1', 117.0, 1),
 ('Ford Sierra ', ' 2', 87.0, 2),
 ('ИЖ Ода ', ' 3', 100.0, 3),
 ('Nissan Pulsar ', ' 4', -91.0, 1),
 ('Nissan Pulsar ', ' 5', -126.0, 2),
 ('Lada 21063 ', ' 6', 89.0, 3),
 ('Ford Sierra ', ' 7', 101.0, 1),
 ('Daewoo Nexia ', ' 8', 99.0, 2),
 ('Nissan Pulsar ', ' 9', 109.0, 3),
 ('Москвич 412 ', ' 10', 109.0, 1),
 ('Peugeot 206 ', ' 11', -95.0, 2),
 ('ГАЗ 21 ', ' 12', 114.0, 3),
 ('Москвич 412 ', ' 13', 95.0, 1),
 ('Nissan Pulsar ', ' 14', 100.0, 2),
 ('Nissan Pulsar ', ' 15', -95.0, 3),
 ('Ford Sierra ', ' 16', 102.0, 1),
 ('Mercedes W201 ', ' 17', 138.0, 2),
 ('Nissan Pulsar ', ' 18', 138.0, 3),
 ('Ford Sierra ', ' 19', 136.0, 1),
 ('Lada 21063 ', ' 20', 125.0, 2),
 ('Ford Sierra ', ' 21', 121.0, 3),
 ('Ford Sierra ', ' 22', 105.0, 1),
 ('Mercedes W201 ', ' 23', 136.0, 2),
 ('Daewoo Nexia ', ' 24', 105.0, 3),
 ('Nissan Pulsar ', ' 25', 145.0, 1),
 ('Peugeot 206 ', ' 26', 113.0, 2),
 ('Москвич 412 ', ' 27', 111.0, 3),
 ('Daewoo Nexia ', ' 28', 152.0, 

# Получение данных

Получите все данные из таблицы сотрудников

In [26]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
      cursor.execute('SELECT * FROM Workers')
      data = cursor.fetchall()
data

[(1, 'Андрей', 20, 'холост'),
 (2, 'Никита', 25, 'женат'),
 (3, 'Иван', 35, 'холост'),
 (4, 'Андрей', 20, 'холост'),
 (5, 'Никита', 25, 'женат'),
 (6, 'Иван', 35, 'холост')]

Получите все данные из таблицы продаж

In [27]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
      cursor.execute('SELECT * FROM Sales')
      data = cursor.fetchall()
data

[(1, 'Lada 21063 ', ' 1', 117.0, 1),
 (2, 'Ford Sierra ', ' 2', 87.0, 2),
 (3, 'ИЖ Ода ', ' 3', 100.0, 3),
 (4, 'Nissan Pulsar ', ' 4', -91.0, 1),
 (5, 'Nissan Pulsar ', ' 5', -126.0, 2),
 (6, 'Lada 21063 ', ' 6', 89.0, 3),
 (7, 'Ford Sierra ', ' 7', 101.0, 1),
 (8, 'Daewoo Nexia ', ' 8', 99.0, 2),
 (9, 'Nissan Pulsar ', ' 9', 109.0, 3),
 (10, 'Москвич 412 ', ' 10', 109.0, 1),
 (11, 'Peugeot 206 ', ' 11', -95.0, 2),
 (12, 'ГАЗ 21 ', ' 12', 114.0, 3),
 (13, 'Москвич 412 ', ' 13', 95.0, 1),
 (14, 'Nissan Pulsar ', ' 14', 100.0, 2),
 (15, 'Nissan Pulsar ', ' 15', -95.0, 3),
 (16, 'Ford Sierra ', ' 16', 102.0, 1),
 (17, 'Mercedes W201 ', ' 17', 138.0, 2),
 (18, 'Nissan Pulsar ', ' 18', 138.0, 3),
 (19, 'Ford Sierra ', ' 19', 136.0, 1),
 (20, 'Lada 21063 ', ' 20', 125.0, 2),
 (21, 'Ford Sierra ', ' 21', 121.0, 3),
 (22, 'Ford Sierra ', ' 22', 105.0, 1),
 (23, 'Mercedes W201 ', ' 23', 136.0, 2),
 (24, 'Daewoo Nexia ', ' 24', 105.0, 3),
 (25, 'Nissan Pulsar ', ' 25', 145.0, 1),
 (26, 'Peugeot

# Анализ имеющихся данных

Посмотрите, попробуйте разные запросы (в т.ч. группировку по моделям, продавцам). Попробуйте какие-нибудь выводы.

Примеры:

Выведите среднее значение результата продаж

In [28]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
                       SELECT  AVG (trade_result) as avg_result FROM SALES
                       ''')
        data = cursor.fetchall()
data

[(154.62,)]

Выведите среднее значение результата продаж для каждого работника в отдельности

In [34]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
                       SELECT   worker_id,AVG (trade_result) as avg_result FROM SALES GROUP BY worker_id

                       ''')
        data = cursor.fetchall()
data

[(1, 175.5), (2, 129.86), (3, 158.5)]

Выведите среднее значение результата продаж для каждого работника в отдельности, и для каждого результата выведите данные о работнике из таблицы `Workers`

In [39]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
                      SELECT  AVG(trade_result) as avg_result, Workers.* FROM Sales JOIN Workers ON Workers.id = Sales.worker_id GROUP BY worker_id
                      ''')
        data = cursor.fetchall()
data

[(175.5, 1, 'Андрей', 20, 'холост'),
 (129.86, 2, 'Никита', 25, 'женат'),
 (158.5, 3, 'Иван', 35, 'холост')]

# Построение модели прогнозирования средствами SQL

Собственник предприятия хотел бы выбрать наиболее перспективного сотрудника. Показатель успешности - вероятное значение продаж на следующий период.

Спрогнозируйте следующее значение продажи каждого сотрдуника. Используйте оконное скользящее среднее (поэкспериментируйте с размером окна).

In [50]:
with contextlib.closing(sqlite3.connect('dealership.db')) as connection:
    with contextlib.closing(connection.cursor()) as cursor:
        cursor.execute('''
                      WITH SalesData AS (
                SELECT
                    worker_id,
                    trade_result,
                    ROW_NUMBER() OVER (PARTITION BY worker_id ) AS row_num
                FROM Sales
            )
            SELECT
                worker_id,
                AVG(trade_result) OVER (PARTITION BY worker_id ORDER BY row_num ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING) AS forecasted_sales
            FROM SalesData
                      ''')

        data = cursor.fetchall()
data

[(1, None),
 (1, 117.0),
 (1, -91.0),
 (1, 101.0),
 (1, 109.0),
 (1, 95.0),
 (1, 102.0),
 (1, 136.0),
 (1, 105.0),
 (1, 145.0),
 (1, 152.0),
 (1, -129.0),
 (1, 151.0),
 (1, 116.0),
 (1, 112.0),
 (1, 141.0),
 (1, 118.0),
 (1, 170.0),
 (1, 138.0),
 (1, 125.0),
 (1, 184.0),
 (1, 172.0),
 (1, 172.0),
 (1, 179.0),
 (1, 153.0),
 (1, 201.0),
 (1, 160.0),
 (1, 173.0),
 (1, 181.0),
 (1, 171.0),
 (1, 163.0),
 (1, 193.0),
 (1, 175.0),
 (1, 180.0),
 (1, -226.0),
 (1, 234.0),
 (1, 221.0),
 (1, 205.0),
 (1, 185.0),
 (1, 209.0),
 (1, 220.0),
 (1, 216.0),
 (1, 263.0),
 (1, 234.0),
 (1, -209.0),
 (1, 260.0),
 (1, 225.0),
 (1, -261.0),
 (1, 252.0),
 (1, 270.0),
 (1, 245.0),
 (1, 265.0),
 (1, 262.0),
 (1, 224.0),
 (1, 306.0),
 (1, 235.0),
 (1, 289.0),
 (1, 249.0),
 (1, 303.0),
 (1, 295.0),
 (1, -228.0),
 (1, 298.0),
 (1, 287.0),
 (1, 336.0),
 (1, 316.0),
 (1, -314.0),
 (1, 264.0),
 (1, 242.0),
 (1, -326.0),
 (1, 288.0),
 (1, 262.0),
 (1, 355.0),
 (1, 364.0),
 (1, -348.0),
 (1, 303.0),
 (1, 314.0),
 (1, 3

# Выводы

В ходе данной лабораторной работы, я получила навыки работы с SQLite. По ходу работы я прочитала данные из Excel, создала по ним базу данных и таблицы "Workers"и "Sales". Затем вставила в таблицы данные и научились получать их с помощью запросов. Также повыводили раздичные запросы по среднему значению результата продаж и  попыталась построить модель прогнозирования  средствами SQL. Данные знания пригодятся для выполнения следующих работ.